## boxes problem
imagine a plain which has two axis y,x 

in this problem we have N boxes that are aligned in one row (y=0) and then we have N objects that are cluttered in this plain with various coordination 

* We want to find an arrangement of objects such that:
    * each box contains exactly one object,
    * each object is stored in one box,
    * the total distance from object $j$ to its storage box is minimal. 

first we have to provide our data

In [1]:
N=15
box_range=range(1,N+1)
object_range=range(1,N+1)



#each box coordination
b_cor={b:(b*12+2,2) for b in box_range}







obj_coords= {1: (140, 6), 2: (146, 8), 3: (132, 14), 4: (53, 28), 
             5: (146, 4), 6: (137, 13), 7: (95, 12), 8: (68, 9), 9: (102, 18), 
             10: (116, 8), 11: (19, 29), 12: (89, 15), 13: (141, 4), 14: (29, 4), 15: (4, 28)}

# we are going to use Euclidean distance to compute the distance between an object and its assigned box
import math
distance={}
for o in object_range:
    for b in box_range:
        dx=obj_coords[o][0]-b_cor[b][0]
        dy=obj_coords[o][1]-b_cor[b][1]
        d=dx*dx+dy*dy
        d_sq=math.sqrt(d)
        distance[b,o]=round(d_sq,1)

In [2]:
# now we have to import the relevant libraries
# since we want to solve this problem with cplex we should import the following libraries
import cplex
import docplex.mp
from docplex.mp.model import Model

In [3]:
# time to make instance of our model
model=Model(name='box_model')

In [4]:
# let's make our variables which are binary variables
x={}
for o in object_range:
    for b in box_range:
        x[b,o]=model.binary_var()

In [8]:
# time to add our constraints

#for each object we have to assign one box
for o in object_range:
    model.add_constraint(model.sum(x[b,o] for b in box_range)==1)

#for each box we have to assign one object    
for b in box_range:
    model.add_constraint(model.sum(x[b,o] for o in object_range)==1)
    

In [10]:
model.print_information()

Model: box_model
 - number of variables: 225
   - binary=225, integer=0, continuous=0
 - number of constraints: 30
   - linear=30
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [11]:
model.minimize(model.sum(distance[b,o]*x[b,o] for o in object_range for b in box_range))

In [12]:
result=model.solve()

In [13]:
model.print_solution()

objective: 269.600
  x11=1
  x29=1
  x40=1
  x49=1
  x73=1
  x90=1
  x97=1
  x110=1
  x128=1
  x144=1
  x153=1
  x171=1
  x192=1
  x197=1
  x211=1


In [30]:
x_key_list=list(x.keys())

In [38]:
x_key_list[0][0]

1

In [33]:
var_list=[11,29,40,49,73,90,97,110,128,144,153,171,192,197,211]

In [42]:
for i in var_list:
    print('we should assign box {} to object {}'.format(x_key_list[i-1][0],x_key_list[i-1][1]))
print('finally the minimum objective value is: ',269.6)    

we should assign box 11 to object 1
we should assign box 14 to object 2
we should assign box 10 to object 3
we should assign box 4 to object 4
we should assign box 13 to object 5
we should assign box 15 to object 6
we should assign box 7 to object 7
we should assign box 5 to object 8
we should assign box 8 to object 9
we should assign box 9 to object 10
we should assign box 3 to object 11
we should assign box 6 to object 12
we should assign box 12 to object 13
we should assign box 2 to object 14
we should assign box 1 to object 15
finally the minimum objective value is:  269.6


## now it is time to make our problem a little trickey

* First, we solve the problem described, and then we add two new constraints and examine how the cost (and solution) changes.
    * From the first solution, we impose that object #1 is assigned to the box immediately to the left of object #2.
    

In [45]:
#add the first constraints
#since we want to impost that the object 1 should be to the immediate left of 
#the object 2 so object 2 can't be assigned to box 1
model.add_(x[1,2]==0)

# now it's time to add that constraints
model.add_constraints(x[n-1,1]>=x[n,2] for n in range(2,N+1))


[docplex.mp.LinearConstraint[](x1,GE,x17),
 docplex.mp.LinearConstraint[](x2,GE,x18),
 docplex.mp.LinearConstraint[](x3,GE,x19),
 docplex.mp.LinearConstraint[](x4,GE,x20),
 docplex.mp.LinearConstraint[](x5,GE,x21),
 docplex.mp.LinearConstraint[](x6,GE,x22),
 docplex.mp.LinearConstraint[](x7,GE,x23),
 docplex.mp.LinearConstraint[](x8,GE,x24),
 docplex.mp.LinearConstraint[](x9,GE,x25),
 docplex.mp.LinearConstraint[](x10,GE,x26),
 docplex.mp.LinearConstraint[](x11,GE,x27),
 docplex.mp.LinearConstraint[](x12,GE,x28),
 docplex.mp.LinearConstraint[](x13,GE,x29),
 docplex.mp.LinearConstraint[](x14,GE,x30)]

In [46]:
model.print_information()

Model: box_model
 - number of variables: 225
   - binary=225, integer=0, continuous=0
 - number of constraints: 46
   - linear=46
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [47]:
results2=model.solve()

In [49]:
model.print_solution()

objective: 269.600
  x14=1
  x30=1
  x40=1
  x49=1
  x73=1
  x86=1
  x97=1
  x110=1
  x128=1
  x144=1
  x153=1
  x171=1
  x192=1
  x197=1
  x211=1


In [52]:
var_list2=[14,30,40,49,73,86,97,110,128,144,153,171,192,197,211]

In [53]:
for i in var_list2:
    print('we should assign box {} to object {}'.format(x_key_list[i-1][0],x_key_list[i-1][1]))
print('finally the minimum objective value is: ',269.6)   

we should assign box 14 to object 1
we should assign box 15 to object 2
we should assign box 10 to object 3
we should assign box 4 to object 4
we should assign box 13 to object 5
we should assign box 11 to object 6
we should assign box 7 to object 7
we should assign box 5 to object 8
we should assign box 8 to object 9
we should assign box 9 to object 10
we should assign box 3 to object 11
we should assign box 6 to object 12
we should assign box 12 to object 13
we should assign box 2 to object 14
we should assign box 1 to object 15
finally the minimum objective value is:  269.6


# finish